# Análise Comparativa de Modelos

## Visão Geral do Notebook

Neste notebook, buscamos prever a eficiência do sono utilizando o conjunto de dados **Sleep Efficiency** disponível no **[Kaggle](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency/data)**. A metodologia seguiu etapas de obtenção, preparação e modelagem dos dados, começando pela limpeza e transformação, incluindo imputação de valores faltantes, codificação de variáveis categóricas, normalização de variáveis numéricas e tratamento de outliers.

Em seguida, treinamos e comparamos quatro modelos preditivos: Regressão Linear, Random Forest, Support Vector Regressor (SVR) e K-Nearest Neighbors (KNN), usando validação cruzada para garantir a robustez dos resultados. Este estudo visa identificar o modelo mais eficaz para prever a eficiência do sono com base em características relacionadas ao estilo de vida e saúde dos indivíduos.

## Metodologia

Neste estudo, o objetivo é prever a eficiência do sono com base em diversas características relacionadas ao estilo de vida e saúde dos indivíduos. O conjunto de dados utilizado, intitulado **Sleep Efficiency** presente na plataforma **[Kaggle](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency/data)**, foi obtido para fins de análise exploratória e modelagem preditiva. A metodologia adotada foi estruturada em várias etapas: **obtenção de dados**, **padronização de variávei**s, **preparação dos dados**, **modelagem preditiva** e **validação dos modelos**.

**Obtenção de Dados:**

Nesta etapa, carregamos e exploramos o conjunto de dados **Sleep Efficiency** para conhecer suas principais características. O dataset inclui informações sobre a eficiência do sono dos indivíduos, bem como variáveis relacionadas ao estilo de vida, como consumo de cafeína, atividades físicas, qualidade do sono, entre outros fatores. Usamos essas informações para prever a eficiência do sono.

**Padronização de Variáveis:**

Nesta etapa, padronizamos o DataFrame convertendo variáveis categóricas para o tipo `category`, transformando colunas de data e hora para o tipo datetime e criando novas colunas separadas para data e hora, e convertendo valores de consumo de álcool de onças para gramas. Essas mudanças organizam e preparam os dados para uma análise mais eficiente.

**Preparação de Dados:**

* A preparação dos dados envolve a limpeza e transformação necessárias para garantir que os dados estejam prontos para a modelagem. Isso inclui:

  - **Tratamento de dados faltantes:** As colunas com valores faltantes foram tratadas utilizando métodos específicos. Para as variáveis numéricas, como `age`, foram imputados valores faltantes utilizando a média. Para variáveis categóricas, como `Smoking Status`, foi utilizada a imputação pela moda.

  - **Codificação de variáveis:** As variáveis categóricas, como `Smoking Status`, `Exercise Frequency`, e `Alcohol Consumption Frequency`, foram codificadas utilizando o OneHotEncoder do Scikit-Learn, facilitando a aplicação de algoritmos de aprendizado de máquina.

  - **Normalização de variáveis:** As variáveis numéricas foram identificadas e selecionadas para normalização utilizando o StandardScaler do Scikit-Learn. O scaler foi aplicado para transformar as colunas selecionadas, resultando em um DataFrame com valores normalizados.

  - **Tratamento de outilers:** Os outliers nas variáveis numéricas foram tratados utilizando métodos de capping baseados no IQR, limitando os valores dentro de intervalos definidos para garantir que não distorçam a análise.

**Modelagem Preditiva:**
    
* Vários modelos foram treinados para prever a eficiência do sono:

  - **Baseline Model (Regressão Linear):** Um modelo simples que serve como ponto de partida para comparação com modelos mais complexos. Ele utiliza a regressão linear para prever a eficiência do sono com base nas variáveis disponíveis.

  - **Random Forest Regressor:** Um modelo avançado que combina múltiplas árvores de decisão (ensemble) para melhorar a precisão e reduzir o risco de overfitting. Ele é eficaz em dados com muitas variáveis e interações complexas.

  - **Support Vector Regressor (SVR):** Um modelo que tenta encontrar a linha de regressão que minimiza o erro, maximizando a margem entre as predições e os valores reais. Ele é útil em cenários onde os dados têm padrões complexos.

  - **K-Nearest Neighbors Regressor (KNN):** Um modelo que prediz a eficiência do sono com base na média dos valores dos vizinhos mais próximos. É um modelo intuitivo, porém sensível a outliers.

**Validação dos Modelos:**

A validação cruzada com k-fold (k=5) foi usada para garantir resultados mais confiáveis e aplicáveis. Isso permite avaliar cada modelo em diferentes partes do conjunto de dados, evitando depender de apenas uma divisão específica dos dados.

**Este notebook é uma ferramenta prática para aprender e aplicar técnicas de machine learning em problemas relacionados à eficiência do sono. No final, queremos encontrar o melhor modelo para o conjunto de dados **[Sleep Efficiency](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency/data)** e obter insights sobre como otimizar a saúde e o bem-estar com base em padrões de sono.**

### 1. Obtenção de dados

#### 1.1. Configuração do Ambiente: Importação das Bibliotecas Necessárias para Análise e Modelagem

In [1]:
# Importação de bibliotecas

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import seaborn as sns

from IPython.display import display, Markdown

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Neste notebook, foram importadas bibliotecas essenciais para manipulação de dados (pandas, numpy), visualização (matplotlib, seaborn) e construção de modelos de machine learning (Scikit-Learn). As funções do Scikit-Learn incluem ferramentas para preparação dos dados, como imputation, escalonamento e codificação, além de modelos preditivos, como Regressão Logística, K-Nearest Neighbors, SVM, Decision Tree e Random Forest, com técnicas de validação cruzada e GridSearch para otimização.


#### 1.2. Carregamento dos Dados e Inspeção Inicial

In [4]:
# Exibição da descrição do conjunto de dados

display(Markdown(
    "O conjunto de dados, proveniente da plataforma Kaggle, contém informações sobre os padrões de sono de indivíduos, identificados por um 'ID do Sujeito' único. Inclui dados sobre idade, gênero, horário de dormir e acordar, duração do sono, eficiência do sono, porcentagens de sono REM, profundo e leve, número de despertares, consumo de cafeína e álcool antes de dormir, status de fumante e frequência de exercícios.\n\n"
    "A variável dependente é a **Sleep eficciency**. As variáveis preditoras que podem influenciar a eficiência do sono são **ID**, **age**, **gender**, **Bedtime**, **Wakeup time**, **Sleep duration**, **REM sleep percentage**, **Deep sleep percentage**, **Light sleep percentage**, **Awakenings**, **Caffeine consumption**, **Alcohol consumption**, **Smoking status**, **Exercise frequency**."
))

# Carregamento dos dados do arquivo CSV

df = pd.read_csv("../data/raw/data.csv")
display(df.head())

O conjunto de dados, proveniente da plataforma Kaggle, contém informações sobre os padrões de sono de indivíduos, identificados por um 'ID do Sujeito' único. Inclui dados sobre idade, gênero, horário de dormir e acordar, duração do sono, eficiência do sono, porcentagens de sono REM, profundo e leve, número de despertares, consumo de cafeína e álcool antes de dormir, status de fumante e frequência de exercícios.

A variável dependente é a **Sleep eficciency**. As variáveis preditoras que podem influenciar a eficiência do sono são **ID**, **age**, **gender**, **Bedtime**, **Wakeup time**, **Sleep duration**, **REM sleep percentage**, **Deep sleep percentage**, **Light sleep percentage**, **Awakenings**, **Caffeine consumption**, **Alcohol consumption**, **Smoking status**, **Exercise frequency**.

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


#### 1.3. Dicionário de dados

In [6]:
# Carregamento do dicionário de dados

df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df_dict)

display(Markdown(
    "---\n"
    "* O conjunto de dados inclui 15 variáveis, sendo 9 quatitativas discretas, 4 quantitativas contínuas, de 2 qualitativas nominais"
    "\n\n"
))

,variavel,descricao,tipo,subtipo
0,ID,Identificador único do indivíduo testado,quantitativa,discreta
1,Age,"Idade do indivíduo testado, em anos",quantitativa,discreta
2,Gender,Gênero do indivíduo testado,qualitativa,nominal
3,Bedtime,O horário em que o indivíduo testado vai para ...,quantitativa,contínua
4,Wakeup time,O horário em que o indivíduo testado acorda to...,quantitativa,contínua
5,Sleep duration,Quantidade total de tempo em que o indivíduo t...,quantitativa,contínua
6,Sleep efficiency,Medida da proporção de tempo na cama gasto dor...,quantitativa,contínua
7,REM sleep percentage,Percentual do tempo total gasto em sono REM,quantitativa,discreta
8,Deep sleep percentage,Percentual do tempo total gasto em sono profundo,quantitativa,discreta
9,Light sleep percentage,Percentual do tempo total gasto em sono leve,quantitativa,discreta


---
* O conjunto de dados inclui 15 variáveis, sendo 9 quatitativas discretas, 4 quantitativas contínuas, de 2 qualitativas nominais



#### 1.4. Informações gerais do DataFrame

In [8]:
# Resumo das informações do DataFrame

df.info()

display(Markdown(
    "---\n"
    "* O DataFrame contém 452 entradas e 15 colunas. As colunas possuem diferentes tipos de dados: `float64` (6 colunas), `int64` (5 colunas), `object` (4 colunas)."

    "---\n"
    "* Algumas colunas apresentam valores ausentes, como `Caffeine consumption` (25 ausentes), `Awakenings` (20 ausentes), `Alcohol consumption` (14 ausentes) e `Exercise frequency` (6 ausentes)."
    "\n\n"
    
    "\n\n"
))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      452 non-null    int64  
 1   Age                     452 non-null    int64  
 2   Gender                  452 non-null    object 
 3   Bedtime                 452 non-null    object 
 4   Wakeup time             452 non-null    object 
 5   Sleep duration          452 non-null    float64
 6   Sleep efficiency        452 non-null    float64
 7   REM sleep percentage    452 non-null    int64  
 8   Deep sleep percentage   452 non-null    int64  
 9   Light sleep percentage  452 non-null    int64  
 10  Awakenings              432 non-null    float64
 11  Caffeine consumption    427 non-null    float64
 12  Alcohol consumption     438 non-null    float64
 13  Smoking status          452 non-null    object 
 14  Exercise frequency      446 non-null    fl

---
* O DataFrame contém 452 entradas e 15 colunas. As colunas possuem diferentes tipos de dados: `float64` (6 colunas), `int64` (5 colunas), `object` (4 colunas).---
* Algumas colunas apresentam valores ausentes, como `Caffeine consumption` (25 ausentes), `Awakenings` (20 ausentes), `Alcohol consumption` (14 ausentes) e `Exercise frequency` (6 ausentes).





#### 1.5. Estatística descritiva do conjunto de dados

In [9]:
# Visualizar a estatística descritiva do conjunto de dados

df.describe(include='all')

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
count,452.000000,452.000000,452,452,452,452.000000,452.000000,452.000000,452.000000,452.000000,432.000000,427.000000,438.000000,452,446.000000
unique,NaN,NaN,2,424,434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,NaN,Male,2021-03-11 01:00:00,2021-11-25 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN
freq,NaN,NaN,228,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298,NaN
mean,226.500000,40.285398,NaN,NaN,NaN,7.465708,0.788916,22.615044,52.823009,24.561947,1.641204,23.653396,1.173516,NaN,1.791480
std,130.625419,13.172250,NaN,NaN,NaN,0.866625,0.135237,3.525963,15.654235,15.313665,1.356762,30.202785,1.621377,NaN,1.428134
min,1.000000,9.000000,NaN,NaN,NaN,5.000000,0.500000,15.000000,18.000000,7.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,113.750000,29.000000,NaN,NaN,NaN,7.000000,0.697500,20.000000,48.250000,15.000000,1.000000,0.000000,0.000000,NaN,0.000000
50%,226.500000,40.000000,NaN,NaN,NaN,7.500000,0.820000,22.000000,58.000000,18.000000,1.000000,25.000000,0.000000,NaN,2.000000
75%,339.250000,52.000000,NaN,NaN,NaN,8.000000,0.900000,25.000000,63.000000,32.500000,3.000000,50.000000,2.000000,NaN,3.000000


* O conjunto de dados, contendo 452 registros, oferece informações sobre padrões de sono dos indivíduos. Inclui variáveis como idade, gênero, horários de dormir e acordar, duração e eficiência do sono, e porcentagens de sono REM, profundo e leve.

* Também abrange dados sobre despertares, consumo de cafeína e álcool, status de fumante e frequência de exercícios.

* As estatísticas mostram, por exemplo, que a idade média é de 40.3 anos e a eficiência do sono média é de 7.47 horas.

### 2. Padronização das Variáveis

#### 2.1. Conversão de Variáveis Categóricas

In [10]:
# Conversão De Variaveis Categóricas

df['Gender'] = df['Gender'].astype('category')
df['Smoking status'] = df['Smoking status'].astype('category')

print("Tipos dos dados após conversão: /n")
print(df.dtypes)

Tipos dos dados após conversão: /n
ID                           int64
Age                          int64
Gender                    category
Bedtime                     object
Wakeup time                 object
Sleep duration             float64
Sleep efficiency           float64
REM sleep percentage         int64
Deep sleep percentage        int64
Light sleep percentage       int64
Awakenings                 float64
Caffeine consumption       float64
Alcohol consumption        float64
Smoking status            category
Exercise frequency         float64
dtype: object


* O processo realiza a conversão das colunas `Gender` e `Smoking status` de um DataFrame para o tipo de dado `category`, que é mais eficiente para variáveis categóricas com um número limitado de valores distintos.

* Após a conversão, é exibido o tipo de dados das colunas do DataFrame para confirmar as mudanças. 

* No fim, essas colunas são listadas como `category`, enquanto as demais mantêm seus tipos originais, como `int64` para inteiros e `float64` para números de ponto flutuante, refletindo a atualização do DataFrame.

#### 2.2. Conversão e Separação de Variáveis de Tempo

In [11]:
# Transformação De Variaveis Temporais

df['Bedtime'] = pd.to_datetime(df['Bedtime'])
df['Wakeup time'] = pd.to_datetime(df['Wakeup time'])

# Separando em data e hora para 'Bedtime'
df['bedtime_date'] = df['Bedtime'].dt.date
df['bedtime_hour'] = df['Bedtime'].dt.time

# Separarando em data e hora para 'Wakeup time'
df['wakeup_date'] = df['Wakeup time'].dt.date
df['wakeup_hour'] = df['Wakeup time'].dt.time

df = df.drop(columns=['Bedtime', 'Wakeup time'])

display(df.head())

,ID,Age,Gender,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency,bedtime_date,bedtime_hour,wakeup_date,wakeup_hour
0,1,65,Female,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0,2021-03-06,01:00:00,2021-03-06,07:00:00
1,2,69,Male,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0,2021-12-05,02:00:00,2021-12-05,09:00:00
2,3,40,Female,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0,2021-05-25,21:30:00,2021-05-25,05:30:00
3,4,40,Female,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0,2021-11-03,02:30:00,2021-11-03,08:30:00
4,5,57,Male,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0,2021-03-13,01:00:00,2021-03-13,09:00:00


* O processo converte as colunas `Bedtime` e `Wakeup time` para o tipo datetime, separa essas informações em novas colunas de data e hora, e exclui as colunas originais.

* O resultado é um DataFrame que organiza melhor os dados temporais.

#### 2.3. Conversão de Unidades: Onças para Gramas na Coluna de `Alcohol consumption`

In [12]:
# Conversão De Unidades

def oz_to_g(oz):
    return oz * 28.3495

# Aplicando a conversão na coluna 'Alcohol consumption'
df['alcohol_consumption_g'] = df['Alcohol consumption'].apply(oz_to_g)
df = df.drop(columns=['Alcohol consumption'])

display(df.head().round(2))

,ID,Age,Gender,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Smoking status,Exercise frequency,bedtime_date,bedtime_hour,wakeup_date,wakeup_hour,alcohol_consumption_g
0,1,65,Female,6.0,0.88,18,70,12,0.0,0.0,Yes,3.0,2021-03-06,01:00:00,2021-03-06,07:00:00,0.00
1,2,69,Male,7.0,0.66,19,28,53,3.0,0.0,Yes,3.0,2021-12-05,02:00:00,2021-12-05,09:00:00,85.05
2,3,40,Female,8.0,0.89,20,70,10,1.0,0.0,No,3.0,2021-05-25,21:30:00,2021-05-25,05:30:00,0.00
3,4,40,Female,6.0,0.51,23,25,52,3.0,50.0,Yes,1.0,2021-11-03,02:30:00,2021-11-03,08:30:00,141.75
4,5,57,Male,8.0,0.76,27,55,18,3.0,0.0,No,3.0,2021-03-13,01:00:00,2021-03-13,09:00:00,85.05


* O processo converte valores da coluna `Alcohol consumption` de onças (oz) para gramas (g) usando a função `oz_to_g`, que multiplica os valores por 28.3495. A conversão é aplicada na coluna e o resultado é armazenado em uma nova coluna `alcohol_consumption_g`.

* Em seguida, a coluna original `Alcohol consumption` é removida do DataFrame. O resultado é um DataFrame com os valores de consumo de álcool em gramas, tornando a unidade de medida mais apropriada para análise.

### 3. Preparação de dados

#### 3.1. Preparação das Variáveis para Análise

In [13]:
# Definição da variável alvo

target_column = 'Sleep efficiency'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
Y = df[target_column]

* No processo de definição da variável alvo, identificou-se a coluna 'Sleep efficiency' como a variável dependente a ser prevista. As colunas nominais e contínuas foram separadas com base no dicionário de variáveis (`df_dict`), sendo que as variáveis nominais são aquelas com tipo 'nominal' e não incluem a variável alvo, enquanto as variáveis contínuas têm o tipo 'continua'.

* Em seguida, o conjunto de dados foi preparado para a modelagem, com `X` contendo todas as colunas, exceto a variável alvo, e `Y` armazenando a coluna 'Sleep efficiency', que será a variável alvo para a previsão.

#### 3.2. Tratamento de dados discrepantes

In [14]:
# Tratamento de dados discrepantes

nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

* Para o tratamento de dados discrepantes, foram definidos dois pipelines de pré-processamento: um para variáveis nominais e outro para variáveis contínuas.

* O pipeline de variáveis nominais inclui a imputação de valores faltantes com a moda, codificação das variáveis com OneHotEncoder (excluindo a primeira categoria para evitar a multicolinearidade) e normalização com StandardScaler.

* O pipeline de variáveis contínuas realiza a imputação de valores faltantes pela média e também a normalização com StandardScaler.

* Ambos os pipelines foram combinados em um `ColumnTransformer`, que aplica o pré-processamento adequado a cada grupo de colunas conforme suas características.

In [15]:
# Aplicando o pipeline de pré-processamento aos dados X

X_transformed = preprocessor.fit_transform(X)

# Verificando se ainda existem dados faltantes

np.isnan(X_transformed).sum()

0

* O pipeline de pré-processamento foi aplicado ao conjunto de dados `X`, processando as variáveis nominais e contínuas conforme as etapas previamente estabelecidas.

* Após a transformação, foi confirmada a ausência de dados faltantes, com a soma de valores `NaN` resultando em `0`.